In [1]:
import time
import copy
import warnings
from typing import Callable

import numpy as np
import pandas as pd
import plotly.io as pio

import Config.constants as cnst
from GazeDetectors.EngbertDetector import EngbertDetector
from DataSetLoaders.DataSetFactory import DataSetFactory

import Analysis.comparisons as cmps
import Analysis.figures as figs

pio.renderers.default = "notebook"

In [2]:
DATASET_NAME = "Lund2013"

LAMBDA = "λ"
ITERATION = "Iteration"
NUM_ITERATIONS = 5

In [ ]:
engbert = EngbertDetector()
lund_dataset = DataSetFactory.load(DATASET_NAME)

In [23]:
start = time.time()

prev_gaze_data = lund_dataset
indexers = [col for col in DataSetFactory._INDEXERS if col in prev_gaze_data.columns]

results = {}
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i in range(1, NUM_ITERATIONS + 1):
        prev_gaze_data = prev_gaze_data.copy()
        for trial_num in prev_gaze_data[cnst.TRIAL].unique():
            trial_data = prev_gaze_data[prev_gaze_data[cnst.TRIAL] == trial_num]
            key = tuple(trial_data[indexers].iloc[0].to_list() + [i])
            res = engbert.detect(
                t=trial_data[cnst.T].to_numpy(),
                x=trial_data[cnst.X].to_numpy(),
                y=trial_data[cnst.Y].to_numpy()
            )
            results[key] = copy.deepcopy(res)

            # nullify detected saccades
            detected_event_labels = res[cnst.GAZE][cnst.EVENT]
            saccade_idxs = trial_data.index[detected_event_labels == cnst.EVENT_LABELS.SACCADE]
            prev_gaze_data.loc[saccade_idxs, cnst.X] = np.nan
            prev_gaze_data.loc[saccade_idxs, cnst.Y] = np.nan

iteration_events = pd.Series({k: [e for e in v[cnst.EVENTS] if e.event_label != cnst.EVENT_LABELS.BLINK]
                              for k, v in results.items()}).sort_index()
iteration_events.index.names = indexers + ["Iteration"]
iteration_events = iteration_events.to_frame()

end = time.time()
print(f"Finished Multi-Iteration Detection:\t{end - start:.2f} seconds")
del start, end

Finished Multi-Iteration Detection:	73.10 seconds


In [4]:
def calc_per_iteration(dataset: pd.DataFrame, func: Callable, **kwargs) -> pd.DataFrame:
    group_by = kwargs.pop("group_by", [cnst.STIMULUS, ITERATION])
    computed = func(dataset, group_by=group_by, **kwargs)
    computed.drop(index="all", inplace=True)  # drop the "all" index because it aggregates over iterations
    computed.index = pd.MultiIndex.from_tuples(computed.index, names=group_by)
    computed_all_stim = computed.groupby(level=ITERATION).agg("sum")
    computed_all_stim.index = pd.MultiIndex.from_tuples([("all", idx) for idx in computed_all_stim.index],
                                                        names=group_by)
    computed = pd.concat([computed, computed_all_stim])[0].unstack(level=ITERATION)
    return computed

### Label Counts

In [24]:
label_counts = calc_per_iteration(iteration_events, cmps.label_counts)
label_counts_fig = figs.count_grid(label_counts,
                                   title="Event Counts per Iteration")
label_counts_fig.show()

### Saccade Amplitude

In [25]:
saccade_amplitudes = calc_per_iteration(iteration_events,
                                        cmps.event_features,
                                        feature=cnst.AMPLITUDE,
                                        ignore_events=[v for v in cnst.EVENT_LABELS if v != cnst.EVENT_LABELS.SACCADE])
saccade_amplitudes_fig = figs.distributions_grid(saccade_amplitudes,
                                                 plot_type="violin",
                                          title="Saccade Amplitudes per Iteration")
saccade_amplitudes_fig.show()

### Saccade Duration

In [26]:
saccade_durations = calc_per_iteration(iteration_events,
                                       cmps.event_features,
                                       feature=cnst.DURATION,
                                       ignore_events=[v for v in cnst.EVENT_LABELS if v != cnst.EVENT_LABELS.SACCADE])
saccade_durations_fig = figs.distributions_grid(saccade_durations,
                                                plot_type="violin",
                                                title="Saccade Durations per Iteration")
saccade_durations_fig.show()

### Saccade Peak-Velocity

In [27]:
saccade_peak_velocities = calc_per_iteration(iteration_events,
                                             cmps.event_features,
                                             feature="peak velocity",
                                             ignore_events=[v for v in cnst.EVENT_LABELS if v != cnst.EVENT_LABELS.SACCADE])
saccade_peak_vel_fig = figs.distributions_grid(saccade_peak_velocities,
                                               plot_type="violin",
                                               title="Saccade Peak Velocities per Iteration")
saccade_peak_vel_fig.show()

C:\Users\jonathanni\PycharmProjects\EyeTrackingAlgroithms\GazeEvents\BaseGazeEvent.py:90: RuntimeWarning:

All-NaN slice encountered



### Micro-Saccade Ration

In [29]:
MICROSACCADE_MAX_AMPLITUDE = 1.0
saccade_counts = iteration_events.map(lambda cell: len([e for e in cell if e.event_label == cnst.EVENT_LABELS.SACCADE]))
microsaccade_counts = iteration_events.map(lambda cell: len([e for e in cell if
                                                             e.event_label == cnst.EVENT_LABELS.SACCADE and
                                                             e.amplitude <= MICROSACCADE_MAX_AMPLITUDE]))
microsaccade_ratio = microsaccade_counts / saccade_counts
microsaccade_ratio = microsaccade_ratio.groupby(level=[cnst.STIMULUS, ITERATION]).agg(list).map(lambda cell: [e for e in cell if pd.notnull(e)])[0]
microsaccade_ratio_all_stim = microsaccade_ratio.groupby(level=ITERATION).agg("sum")
microsaccade_ratio_all_stim.index = pd.MultiIndex.from_tuples(
    [("all", idx) for idx in microsaccade_ratio_all_stim.index],
    names=[cnst.STIMULUS, ITERATION])
microsaccade_ratio = pd.concat([microsaccade_ratio, microsaccade_ratio_all_stim]).unstack(level=ITERATION)
microsaccade_ratio_fig = figs.distributions_grid(microsaccade_ratio,
                                                 plot_type="violin",
                                                 # limit_pdf=True,
                                                 title="Microsaccade Ratio per Iteration")
microsaccade_ratio_fig.show()